In [5]:
!pip install nltk rouge
!git clone https://github.com/salaniz/pycocoevalcap

fatal: destination path 'pycocoevalcap' already exists and is not an empty directory.


In [6]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import json
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import single_meteor_score
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize

# Load JSON data
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

data = load_json('/content/drive/My Drive/MasterThesis/flickr8k_dataset/combined_captions_data_flickr_final.json')

# Ensure NLTK's resources are downloaded
nltk.download('punkt')
nltk.download('wordnet')


def calculate_meteor(reference, candidate):
    if candidate.strip() == "":
        return 0  # Return 0 if candidate is blank
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

# Function to calculate BLEU score with multiple references
def calculate_corpus_bleu(references, candidate):
    if candidate.strip() == "":
        return 0  # Return 0 if candidate is blank
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

# Initialize ROUGE scorer
rouge = Rouge()

# Prepare to accumulate scores for averaging
model_scores = {
    'blip': {'bleu': [], 'meteor': [], 'rouge': []},
    'gpt2': {'bleu': [], 'meteor': [], 'rouge': []},
    'pix2struct': {'bleu': [], 'meteor': [], 'rouge': []}
}

# Calculate scores for each model
for item in data.values():
    original_captions = item['original_coco_captions']
    for model_name, generated_caption in item['generated_captions'].items():
        if model_name == "pix2struct" and generated_caption.strip() == "":
            #print(f"Skipping {model_name} for image_id {item_id} due to blank caption.")
            continue
        bleu_score = calculate_corpus_bleu(original_captions, generated_caption)
        meteor = calculate_meteor(original_captions[0], generated_caption)
        rouge_scores = rouge.get_scores(generated_caption, original_captions[0])[0]

        model_scores[model_name]['bleu'].append(bleu_score)
        model_scores[model_name]['meteor'].append(meteor)
        model_scores[model_name]['rouge'].append(rouge_scores)

# Print average scores for each model
for model_name, scores in model_scores.items():
    avg_bleu = sum(scores['bleu']) / len(scores['bleu']) if scores['bleu'] else 0
    avg_meteor = sum(scores['meteor']) / len(scores['meteor']) if scores['meteor'] else 0

    # Initialize sums for ROUGE scores
    sum_rouge_1f = sum_rouge_2f = sum_rouge_lf = 0

    for rouge_score in scores['rouge']:
        sum_rouge_1f += rouge_score['rouge-1']['f']
        sum_rouge_2f += rouge_score['rouge-2']['f']
        sum_rouge_lf += rouge_score['rouge-l']['f']

    len_rouge = len(scores['rouge']) if scores['rouge'] else 1  # Avoid division by zero
    avg_rouge_1f = sum_rouge_1f / len_rouge
    avg_rouge_2f = sum_rouge_2f / len_rouge
    avg_rouge_lf = sum_rouge_lf / len_rouge

    print(f"Model: {model_name}")
    print(f"Average BLEU: {avg_bleu:.4f}")
    print(f"Average METEOR: {avg_meteor:.4f}")
    print(f"Average ROUGE-1 F: {avg_rouge_1f:.4f}")
    print(f"Average ROUGE-2 F: {avg_rouge_2f:.4f}")
    print(f"Average ROUGE-L F: {avg_rouge_lf:.4f}\n")

# Print average scores for each model
for model_name, scores in model_scores.items():
    avg_bleu = sum(scores['bleu']) / len(scores['bleu']) if scores['bleu'] else 0
    avg_meteor = sum(scores['meteor']) / len(scores['meteor']) if scores['meteor'] else 0

    # Initialize sums for ROUGE scores
    sum_rouge_1f = sum_rouge_2f = sum_rouge_lf = 0

    for rouge_score in scores['rouge']:
        sum_rouge_1f += rouge_score['rouge-1']['f']
        sum_rouge_2f += rouge_score['rouge-2']['f']
        sum_rouge_lf += rouge_score['rouge-l']['f']

    len_rouge = len(scores['rouge']) if scores['rouge'] else 1  # Avoid division by zero
    avg_rouge_1f = sum_rouge_1f / len_rouge
    avg_rouge_2f = sum_rouge_2f / len_rouge
    avg_rouge_lf = sum_rouge_lf / len_rouge

    print(f"Model: {model_name}")
    print(f"Average BLEU: {avg_bleu:.3f}")
    print(f"Average METEOR: {avg_meteor:.3f}")
    print(f"Average ROUGE-1 F: {avg_rouge_1f:.3f}")
    print(f"Average ROUGE-2 F: {avg_rouge_2f:.3f}")
    print(f"Average ROUGE-L F: {avg_rouge_lf:.3f}\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Model: blip
Average BLEU: 0.1771
Average METEOR: 0.2921
Average ROUGE-1 F: 0.3210
Average ROUGE-2 F: 0.0972
Average ROUGE-L F: 0.2820

Model: gpt2
Average BLEU: 0.1486
Average METEOR: 0.2312
Average ROUGE-1 F: 0.2858
Average ROUGE-2 F: 0.0757
Average ROUGE-L F: 0.2626

Model: pix2struct
Average BLEU: 0.0421
Average METEOR: 0.1540
Average ROUGE-1 F: 0.2150
Average ROUGE-2 F: 0.0269
Average ROUGE-L F: 0.1981

Model: blip
Average BLEU: 0.177
Average METEOR: 0.292
Average ROUGE-1 F: 0.321
Average ROUGE-2 F: 0.097
Average ROUGE-L F: 0.282

Model: gpt2
Average BLEU: 0.149
Average METEOR: 0.231
Average ROUGE-1 F: 0.286
Average ROUGE-2 F: 0.076
Average ROUGE-L F: 0.263

Model: pix2struct
Average BLEU: 0.042
Average METEOR: 0.154
Average ROUGE-1 F: 0.215
Average ROUGE-2 F: 0.027
Average ROUGE-L F: 0.198



In [ ]:
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageTool('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/flickr8k_dataset/Scenario1_distilbart_with_all_similarities_only_2_flickr.json')
data2 = load_json('/content/drive/MyDrive/MasterThesis/flickr8k_dataset/Scenario2_distilbart_with_all_similarities_only_2_flickr.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]
    item2 = data2[key]

    chosen_item = item1 if item1['cosine_similarity_score'] > item2['cosine_similarity_score'] else item2

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))


    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Save to a new JSON file
with open('/content/drive/MyDrive/MasterThesis/comparison_results_with_scores_scenario1_distilbart_only2_flickr.json', 'w') as f:
    json.dump(results, f, indent=4)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Load the computed scores from the JSON file
results = load_json('/content/drive/MyDrive/MasterThesis/comparison_results_with_scores_scenario1_distilbart_only2_flickr.json')

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

Average BLEU Score: 0.1582326971822005
Average METEOR Score: 0.2898552832645758
Average ROUGE-1 F Score: 0.32485324008397753
Average ROUGE-2 F Score: 0.09472287107742738
Average ROUGE-L F Score: 0.28273774923594275


In [ ]:
# Load the computed scores from the JSON file
results = load_json('/content/drive/MyDrive/MasterThesis/comparison_results_with_scores_scenario1_distilbart_only2_corrected_flickr.json')

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

Average BLEU Score: 0.15376051129788784
Average METEOR Score: 0.22931040011291245
Average ROUGE-1 F Score: 0.28055565583312847
Average ROUGE-2 F Score: 0.08475200199692931
Average ROUGE-L F Score: 0.2428897709361035


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python
import re

tool = language_tool_python.LanguageToolPublicAPI('en-US')


def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_flickr.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]

    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.14912915328780385
Average METEOR Score: 0.30576680652545896
Average ROUGE-1 F Score: 0.36559984355694214
Average ROUGE-2 F Score: 0.10582106490282048
Average ROUGE-L F Score: 0.33045508626095244


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python
import re

tool = language_tool_python.LanguageToolPublicAPI('en-US')


def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_flickr.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.14626229755265238
Average METEOR Score: 0.28164445040829417
Average ROUGE-1 F Score: 0.34811117380040524
Average ROUGE-2 F Score: 0.09808170388236706
Average ROUGE-L F Score: 0.3166142058625418


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')
import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_3_flickr.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_from_second_there(sentence):
   # Find all occurrences of "there" and get their indices
    indices = [m.start() for m in re.finditer(r'\bthere\b', sentence, re.IGNORECASE)]
    # If "there" occurs at least twice, trim the sentence right before the second occurrence
    if len(indices) > 1:
        return sentence[:indices[1]].strip()
    # If "there" doesn't occur twice, return the original sentence
    return sentence

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.12303078648357225
Average METEOR Score: 0.23329621045234303
Average ROUGE-1 F Score: 0.3229167554282278
Average ROUGE-2 F Score: 0.08027172103005538
Average ROUGE-L F Score: 0.2965255470020282


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python
import re

tool = language_tool_python.LanguageToolPublicAPI('en-US')


def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_3_flickr.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.17379627254228242
Average METEOR Score: 0.2963026718315558
Average ROUGE-1 F Score: 0.3263599606310959
Average ROUGE-2 F Score: 0.09854798595481058
Average ROUGE-L F Score: 0.28773272445922005


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python
import re

tool = language_tool_python.LanguageToolPublicAPI('en-US')


def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_4_flickr.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.1749347447339221
Average METEOR Score: 0.29725565453123554
Average ROUGE-1 F Score: 0.3298630876598179
Average ROUGE-2 F Score: 0.10038213197608935
Average ROUGE-L F Score: 0.2900652165562599


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')
import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_4_flickr.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.13078153703891623
Average METEOR Score: 0.2441484017307094
Average ROUGE-1 F Score: 0.3437806804963281
Average ROUGE-2 F Score: 0.0876097086850072
Average ROUGE-L F Score: 0.3149291542438197


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')
import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_5_flickr.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence
rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.1292566880974153
Average METEOR Score: 0.24414581069037422
Average ROUGE-1 F Score: 0.3445326651212141
Average ROUGE-2 F Score: 0.08886126775515306
Average ROUGE-L F Score: 0.31451095850137495


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')
import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_5_flickr.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.1756519257173352
Average METEOR Score: 0.2985343085661064
Average ROUGE-1 F Score: 0.3281007866982936
Average ROUGE-2 F Score: 0.09974272874391733
Average ROUGE-L F Score: 0.2886660552915511


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')
import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_2_flickr.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)


    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Average BLEU Score: 0.17276417368852393
Average METEOR Score: 0.2964870071315617
Average ROUGE-1 F Score: 0.3241693731032344
Average ROUGE-2 F Score: 0.09645737247494714
Average ROUGE-L F Score: 0.2857146561628239


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')
import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_2_flickr.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.1170437230458446
Average METEOR Score: 0.23208427256175354
Average ROUGE-1 F Score: 0.3104546446475384
Average ROUGE-2 F Score: 0.0761952202320048
Average ROUGE-L F Score: 0.2849256446160658


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re


def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_ann_flickr_modified.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    print(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')


    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    #rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]
    rouge_scores = 0
    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    #total_rouge_1_f += value['rouge']['rouge-1']['f']
    #total_rouge_2_f += value['rouge']['rouge-2']['f']
    #total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
#average_rouge_1_f = total_rouge_1_f / num_items
#average_rouge_2_f = total_rouge_2_f / num_items
#average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
#print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
#print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
#print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


A little girl standing next to a wooden fence there is a little girl.
Two dogs playing with each other on the street
 There is a little girl that is sitting on the ground in front of a rainbow a
There is a man laying on a bench with a dog next to him.
There is a man wearing a hat and glasses with a hat.
There is a little girl climbing on a rope net in a playground.
 There is a dog that is running in the grass with a Frisbee in it
 There is a dog that is playing with a ball on the beach
 There is a young boy riding a skateboard down the street a little boy standing next
 There is a dog that is jumping over a log in the grass
 A dog that is running in the snow with a Frisbee in its mouth
There are two people standing in the snow with skis.
 There are many people climbing up the side of a rock wall
 A dog running in the grass with a Frisbee is a dog that there is
 There is a white dog running with a yellow ball in its mouth
 Video shows a man running across a dog that is jumping in the ai

In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python
import re

tool = language_tool_python.LanguageToolPublicAPI('en-US')




def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_ann_flickr_modified.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_comma_a_ending(sentence):
    # This pattern matches a comma followed by any characters until the end of the sentence
    pattern = r',.*$'
    # Remove the pattern (i.e., everything after the comma) if found in the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

rouge = Rouge()


results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)
    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)

    print(trimmed_and_corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')


    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


A little girl standing next to a wooden fence there is a little girl.
{'rouge-1': {'r': 0.2, 'p': 0.2727272727272727, 'f': 0.23076922588757406}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.2, 'p': 0.2727272727272727, 'f': 0.23076922588757406}}
Two dogs playing with each other on the street
{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}
 There is a little girl that is sitting on the ground in front of a rainbow a
{'rouge-1': {'r': 0.4375, 'p': 0.5, 'f': 0.46666666168888893}, 'rouge-2': {'r': 0.2222222222222222, 'p': 0.25, 'f': 0.23529411266435996}, 'rouge-l': {'r': 0.4375, 'p': 0.5, 'f': 0.46666666168888893}}
There is a man laying on a bench with a dog next to him.
{'rouge-1': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}, 'rouge-2': {'r': 0.18181818181818182, 'p': 0.15384615384615385, 'f': 0.16666666170138905}, 'rouge-l': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}}
There is a man wea

In [ ]:

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)
# Iterate through the results to accumulate scores
for key, value in results.items():

    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

Average BLEU Score: 0.17972342059589658
Average METEOR Score: 0.3005205012337598
Average ROUGE-1 F Score: 0.34847450291603654
Average ROUGE-2 F Score: 0.10723968559825228
Average ROUGE-L F Score: 0.31062186366762945
